In this notebook, we will try to predict heart disease events by 11 clinical features. We will first use Decision Tree, then try Random Forest and finally XGBoost algorithm to see and compare the accuracy values. To improve the accuracy, we will apply hyperparameter tuning with GridSearchCV

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr, chi2_contingency

RANDOM_STATE = 19 #We will pass it to every sklearn call so we ensure reproducibility

#Attribute Information

*   Age: age of the patient [years]

*   Sex: sex of the patient [M: Male, F: Female]
*   ChestPainType: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
*   RestingBP: resting blood pressure [mm Hg]

*   Cholesterol: serum cholesterol [mm/dl]
*   FastingBS: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
*   RestingECG: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]
*   MaxHR: maximum heart rate achieved [Numeric value between 60 and 202]
*   ExerciseAngina: exercise-induced angina [Y: Yes, N: No]
*   Oldpeak: oldpeak = ST [Numeric value measured in depression]
*   ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
*   HeartDisease: output class [1: heart disease, 0: Normal]

Loading our dataset as DataFrame

In [2]:
# Load the dataset using pandas
df = pd.read_csv("heart.csv")

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


Creating training and test datasets

In [4]:
features = [x for x in df.columns if x not in 'HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(df[features], df['HeartDisease'], train_size = 0.8, random_state = RANDOM_STATE )

print(X_train.shape, y_train.shape,  X_test.shape)

(734, 11) (734,) (184, 11)


# Feature Selection

We will use chi-square method for categorical features with categorical label, and Pearson correlation coefficient for numerical features with categorical labels.

If we would use append method to add a row, dictionary as input would be okay. But for future convenience, I will prefer concat there which accpets DataFrame as new input, append format can be seen in the comment line.

DataFrame can be created via enclosing scalar variables to [ ] because Pandas expects each column to be a sequence (e.g., a list, array, or Series), and using [ ] ensures that our scalar values are treated as sequences with one element.

We spesify ignore_index = True, because we want to sort each row with increasing indices one by one. If we dont spesify that argument, the indexes of the rows will remain same as the time they created, indexes may not be sorted in the final DataFrame. For example, second row of the argument (index 1) will remain same even though if we add that row as fifth one.

In [5]:
categorical_features = ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

chi_square_results = pd.DataFrame(columns=['Feature', 'Chi-Square', 'P-value'])

for feature in categorical_features:
  contingency_table = pd.crosstab(X_train[feature],y_train)
  chi2, p, _, _ = chi2_contingency(contingency_table)
  chi_square_results = pd.concat([chi_square_results, pd.DataFrame({'Feature': [feature], 'Chi-Square': [chi2], 'P-value': [p]})],ignore_index=True)

chi_square_results
#chi_square_results = chi_square_results.append({'Feature': feature, 'Chi-Square': chi2, 'P-value': p}, ignore_index=True)

#if P is close or greater than 0.05 , we will exclude the column from DataFrame

,Feature,Chi-Square,P-value
0,Sex,72.540909,1.636051e-17
1,ChestPainType,223.741152,3.118655e-48
2,FastingBS,53.053967,3.245089e-13
3,RestingECG,8.510570,1.418905e-02
4,ExerciseAngina,185.541268,2.989753e-42
5,ST_Slope,293.418874,1.927086e-64


In [6]:
numerical_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']

correlation_results = pd.DataFrame(columns = ['Feature', 'Correlation', 'P-value'])

for feature in numerical_features:
  correlation, p_value = pearsonr(X_train[feature],y_train)
  correlation_results = pd.concat([correlation_results, pd.DataFrame({'Feature': [feature], 'Correlation': [correlation], 'P-value': [p_value]})],ignore_index=True)


correlation_results
#if P is close or greater than 0.05 , we will exclude the column from DataFrame
#correlation_results = correlation_results.append({'Feature': feature, 'Correlation': correlation, 'P-value': p_value}, ignore_index=True)

,Feature,Correlation,P-value
0,Age,0.269404,1.135778e-13
1,RestingBP,0.074391,4.392590e-02
2,Cholesterol,-0.238975,5.429677e-11
3,MaxHR,-0.418818,1.542613e-32
4,Oldpeak,0.418577,1.688118e-32


We can drop RestingBP since P-value is close to 0.05

In [7]:
X_train = X_train.drop(X_train.columns[[3]], axis= 1)
X_test = X_test.drop(X_test.columns[[3]], axis= 1)
X_train

,Age,Sex,ChestPainType,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
600,57,M,ASY,207,0,ST,96,Y,1.0,Flat
477,61,M,ASY,0,1,Normal,108,Y,2.0,Down
307,53,M,ATA,0,0,ST,120,N,0.7,Down
495,64,F,ASY,276,0,Normal,140,Y,1.0,Flat
542,54,F,ASY,274,0,Normal,105,Y,1.5,Flat
...,...,...,...,...,...,...,...,...,...,...
19,36,M,ATA,267,0,Normal,160,N,3.0,Flat
354,55,M,ASY,0,0,ST,92,N,0.3,Up
757,50,M,NAP,233,0,Normal,163,N,0.6,Flat
622,59,M,ASY,239,0,LVH,142,Y,1.2,Flat


Applying One-Hot Encoding to the categorical features

In [8]:
cat_variables = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

X_train = pd.get_dummies(data = X_train, prefix = cat_variables, columns = cat_variables)
#X_cross_val = pd.get_dummies(data = X_cross_val, prefix = cat_variables, columns = cat_variables)
X_test = pd.get_dummies(data = X_test, prefix = cat_variables, columns = cat_variables)
X_train

,Age,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
600,57,207,0,96,1.0,0,1,1,0,0,0,0,0,1,0,1,0,1,0
477,61,0,1,108,2.0,0,1,1,0,0,0,0,1,0,0,1,1,0,0
307,53,0,0,120,0.7,0,1,0,1,0,0,0,0,1,1,0,1,0,0
495,64,276,0,140,1.0,1,0,1,0,0,0,0,1,0,0,1,0,1,0
542,54,274,0,105,1.5,1,0,1,0,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,36,267,0,160,3.0,0,1,0,1,0,0,0,1,0,1,0,0,1,0
354,55,0,0,92,0.3,0,1,1,0,0,0,0,0,1,1,0,0,0,1
757,50,233,0,163,0.6,0,1,0,0,1,0,0,1,0,1,0,0,1,0
622,59,239,0,142,1.2,0,1,1,0,0,0,1,0,0,0,1,0,1,0


We will use GridSearchCV to find best hyperparameters for our Decision Tree  algorithm. It will automatically do cross-validation, therefore we do not need an additional set for that.

In [9]:
dt_classifier = DecisionTreeClassifier()
parameter_grid = {
    'min_samples_split': [2,5,10,20,30,40,50,60,70,100],
    'max_depth': [1,2,3,4,5,8,16,32],
    'min_samples_leaf': [1,2,3,4,5,8,16,32,64]
}
grid_search = GridSearchCV(dt_classifier, parameter_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
accuracy = best_estimator.score(X_test, y_test)

In [12]:
print(f"best fitted parameters: {best_params}")
print(f"Accuracy of Decision Tree: {accuracy}")

best fitted parameters: {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 30}
Accuracy of Decision Tree: 0.8478260869565217


We got accuracy 0.848. Now, let's try Random Forest algorithm to see whether we will get better results.

In [13]:
rf_classifier = RandomForestClassifier()
param_grid ={
    'min_samples_split': [2,5,10,20, 30, 40, 50, 60, 100, 200, 300],
    'max_depth': [2, 4, 8, 16, 32],
    'n_estimators': [10, 50, 75, 100, 150, 500]
}

grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
accuracy = best_estimator.score(X_test, y_test)

In [15]:
print(best_params)
print(accuracy)
#{'max_depth': 64, 'min_samples_split': 10, 'n_estimators': 50}
#0.8695652173913043

{'max_depth': 8, 'min_samples_split': 2, 'n_estimators': 100}
0.875


We got a bit better result compared to the Decision Tree accuracy. Finally, let's use XGBoost.

We will first try XGBoost() and then use GridSearchCV() to see how much the accuracy differs with additional hyperparameter tuning.

In [16]:
X_train_boost, X_cross_boost, y_train_boost, y_cross_boost = train_test_split(X_train, y_train, train_size = 0.8, random_state = RANDOM_STATE )
print(X_train_boost.shape, X_cross_boost.shape)

(587, 19) (147, 19)


We created additional cross validation set because XGBoost allows us to send an evaluation dataset to apply cost function and prevent overfitting.

XGBoost results in better accuracy compared to Random Forest without using additional tuning in both since it provides internal optimization procedure at each step.

In [17]:
xgb_model = XGBClassifier(n_estimators = 500, learning_rate = 0.1,verbosity = 1, random_state = RANDOM_STATE)
xgb_model.fit(X_train_boost,y_train_boost, eval_set = [(X_cross_boost,y_cross_boost)], early_stopping_rounds = 10)

[0]	validation_0-logloss:0.64159
[1]	validation_0-logloss:0.59796
[2]	validation_0-logloss:0.55984
[3]	validation_0-logloss:0.53017
[4]	validation_0-logloss:0.50371
[5]	validation_0-logloss:0.48321
[6]	validation_0-logloss:0.46366
[7]	validation_0-logloss:0.45032
[8]	validation_0-logloss:0.43522
[9]	validation_0-logloss:0.42209
[10]	validation_0-logloss:0.41064
[11]	validation_0-logloss:0.40021
[12]	validation_0-logloss:0.39233
[13]	validation_0-logloss:0.38534
[14]	validation_0-logloss:0.37702
[15]	validation_0-logloss:0.37326
[16]	validation_0-logloss:0.36837
[17]	validation_0-logloss:0.36413
[18]	validation_0-logloss:0.35956
[19]	validation_0-logloss:0.35525
[20]	validation_0-logloss:0.35348
[21]	validation_0-logloss:0.35234
[22]	validation_0-logloss:0.34968
[23]	validation_0-logloss:0.34827
[24]	validation_0-logloss:0.34779
[25]	validation_0-logloss:0.34592
[26]	validation_0-logloss:0.34354
[27]	validation_0-logloss:0.34201
[28]	validation_0-logloss:0.34220
[29]	validation_0-loglos

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[30]	validation_0-logloss:0.33748
[31]	validation_0-logloss:0.33635
[32]	validation_0-logloss:0.33497
[33]	validation_0-logloss:0.33556
[34]	validation_0-logloss:0.33486
[35]	validation_0-logloss:0.33673
[36]	validation_0-logloss:0.33566
[37]	validation_0-logloss:0.33478
[38]	validation_0-logloss:0.33400
[39]	validation_0-logloss:0.33372
[40]	validation_0-logloss:0.33349
[41]	validation_0-logloss:0.33283
[42]	validation_0-logloss:0.33357
[43]	validation_0-logloss:0.33441
[44]	validation_0-logloss:0.33506
[45]	validation_0-logloss:0.33623
[46]	validation_0-logloss:0.33766
[47]	validation_0-logloss:0.33800
[48]	validation_0-logloss:0.33888
[49]	validation_0-logloss:0.33892
[50]	validation_0-logloss:0.33980


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=19, ...)

XGBoost iterated 10 more time after the best value of loss function

In [18]:
xgb_model.best_iteration

41

In [22]:
print(f"accuracy on training set: {accuracy_score(xgb_model.predict(X_train),y_train)}")
print(f"accuracy score of classic XGBoost: {accuracy_score(xgb_model.predict(X_test),y_test)}")

accuracy on training set: 0.9482288828337875
accuracy score of classic XGBoost: 0.842391304347826


We got 0.842 accuracy from XGBoost. Now we will try with additional tuning.

In [20]:
xgboost = XGBClassifier(verbosity=1)
parameters_grid = {
    'n_estimators': [50,100, 200, 300],
    'max_depth': [3, 4, 5,8,10],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_child_weight': [1, 2, 3]
    # Add more hyperparameters to tune as needed
}
grid_search = GridSearchCV(
    estimator= xgboost,
    param_grid=parameters_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5, 8, 10],
                         'min_child_weight': [1, 2, 3],
                         'n_estimators': [50, 100, 200, 300]},
             scoring='accuracy')

In [21]:
best = grid_search.best_params_
best_estimator = grid_search.best_estimator_
accuracy = best_estimator.score(X_test, y_test)
print(f"best fitted parameters: {best}")
print(f"accuracy score of tuned XGBoost: {accuracy}")

best fitted parameters: {'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 50}
accuracy score of tuned XGBoost: 0.8641304347826086


We got better accuracy result with additional hyperparameter tuning. Additionally, accuracy of training and test sets became closer now, shows less overfitting.

We can also extract the best fitted parameters and train a model ourselve to see that we can get same accuracy value.

In [23]:
xgboost_f = XGBClassifier(**best)
xgb_tuned =  xgboost_f.fit(X_train,y_train)

In [24]:
print(accuracy_score(xgboost_f.predict(X_train),y_train))
print(accuracy_score(xgboost_f.predict(X_test),y_test))

0.9168937329700273
0.8641304347826086
